# Generación de texto con LSTMs

In [1]:
# Paso 1: importar librerías
import re
import numpy as np
import pandas as pd
from sklearn import preprocessing
from keras import models, layers

Using TensorFlow backend.


In [2]:
#Paso 2: importar cuento  + pasar todo a minusculas + pasar caracteres a integers
# load ascii text and covert to lowercase
f = open('MobyDick.txt', 'r', encoding="utf8") 
text = f.read()
f.close()
text = text.lower()
#text = re.sub(r"[^a-zA-Z0-9]+", ' ', text)
#text = text[:100000]

# create mapping of unique chars to integers, and a reverse mapping
chars = np.array(list(text))
encoder = preprocessing.LabelEncoder().fit(chars)
encoded_chars = encoder.transform(chars)

# normalize data
mean = encoded_chars.mean()
stdv = encoded_chars.std()
encoded_chars = (encoded_chars - mean) / stdv

len(chars)

1238244

In [3]:
#paso 3: preparar datos: 
# 1- preprocesar: ejemplo: CHAPT -> E; HAPTE -> R (pero con 100 caracteres)

# split a univariate sequence into samples
def split_sequence(sequence, n_steps, n_step_diff=1):
    X, y = list(), list()
    i = 0
    while i <len(sequence): 
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
        i = i + (n_step_diff)
    return np.array(X), np.array(y)

X_train, y_train = split_sequence(encoded_chars, 100, 100)

# 2- cambiar dataset para que la entrada sea válida: [Samples, time_steps,features]
n_samples = X_train.shape[0]
n_steps = X_train.shape[1] # 100
n_features = 1
X_train = X_train.reshape((n_samples, n_steps, n_features))

# 3-Normalizar
    # normalizamos todo después de usar label encoder

# prepare the dataset of input to output pairs encoded as integers

In [4]:
# Definimos modelo LSTM
lstm_model = models.Sequential([
    layers.LSTM(32, dropout=0.1, recurrent_dropout=0.5, return_sequences=True, input_shape=X_train.shape[-2:]),
    layers.LSTM(32, dropout=0.1, recurrent_dropout=0.5),
    layers.Dense(1)
])

lstm_model.compile(optimizer='rmsprop', loss='mse')

In [5]:
#Hacemos el FIT
history = lstm_model.fit(X_train, y_train, epochs=5, validation_split=0.2)

Train on 9905 samples, validate on 2477 samples
Epoch 1/5
9905/9905 [==============================] - 42s 4ms/step - loss: 0.9562 - val_loss: 0.9709
Epoch 2/5
9905/9905 [==============================] - 40s 4ms/step - loss: 0.9475 - val_loss: 0.9569
Epoch 3/5
9905/9905 [==============================] - 40s 4ms/step - loss: 0.9390 - val_loss: 0.9551
Epoch 4/5
9905/9905 [==============================] - 40s 4ms/step - loss: 0.9390 - val_loss: 0.9548
Epoch 5/5
9905/9905 [==============================] - 41s 4ms/step - loss: 0.9377 - val_loss: 0.9529
